In [64]:
import matplotlib.pyplot as plt
from dataload import *
from gan_architecture import *
from utils import *
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from dataload import *
from gan_architecture import *
import pandas as pd
import numpy as np
import copy

In [65]:
class SuperNetwork(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.lin = nn.Linear(input_size, 4)
        # self.conv = nn.Conv1d(4, 4, 1)
        self.bn = nn.BatchNorm1d(4)
        self.activation = nn.ReLU()

    def forward(self, x):
        x = self.lin(x)
        # x = self.conv(x)
        x = self.bn(x)
        x = self.activation(x)
        return x

In [66]:
bs = 16
input = torch.randn((bs, 4), dtype=torch.float, device='cpu')
solution = torch.full((bs, 4), 3.5, dtype=torch.float, device='cpu')

In [67]:
s = SuperNetwork(4)
s2 = copy.deepcopy(s)
criterion = nn.MSELoss() 
optimizer = optim.Adam(s.parameters(), lr = 1.0)
optimizer2 = optim.Adam(s2.parameters(), lr = 1.0)

In [68]:
print(s.parameters)

<bound method Module.parameters of SuperNetwork(
  (lin): Linear(in_features=4, out_features=4, bias=True)
  (bn): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (activation): ReLU()
)>


In [69]:
gen = s.modules()
next(gen)
m_list  = list(gen)

gen2 = s2.modules()
next(gen2)
m_list2 = list(gen2)


for l in m_list[:-1] :
    print(l.weight.grad)

None
None


In [70]:
loss = criterion(s(input), solution)
loss.backward(retain_graph=True)
print(loss)

tensor(9.9606, grad_fn=<MseLossBackward0>)


In [71]:
loss2 = criterion(s2(input), solution)
loss2.backward()

In [72]:
print(loss)
print(loss2)

tensor(9.9606, grad_fn=<MseLossBackward0>)
tensor(9.9606, grad_fn=<MseLossBackward0>)


In [73]:
for l in m_list[:-1] :
    print(l.weight.grad)

for l in m_list2[:-1] :
    print(l.weight.grad)

# print(s.lin.weight.grad)

tensor([[-0.0882,  0.1290,  0.0674,  0.1725],
        [-0.0785, -0.1051,  0.1533, -0.1014],
        [ 0.0451, -0.0962,  0.1815,  0.0388],
        [ 0.1052, -0.3262,  0.0703, -0.1310]])
tensor([-0.2991, -0.5083, -0.3425, -0.5579])
tensor([[-0.0882,  0.1290,  0.0674,  0.1725],
        [-0.0785, -0.1051,  0.1533, -0.1014],
        [ 0.0451, -0.0962,  0.1815,  0.0388],
        [ 0.1052, -0.3262,  0.0703, -0.1310]])
tensor([-0.2991, -0.5083, -0.3425, -0.5579])


In [74]:
apply_weight_decay(*s.modules(), weight_decay_factor=1, wo_bn=True)

In [75]:
optimizer.step()
optimizer2.step()

In [76]:
for l in m_list[:-1] :
    print(l.weight.grad)

for l in m_list2[:-1] :
    print(l.weight.grad)

tensor([[ 0.6775, -0.4496, -0.0959,  1.0606],
        [-0.8692,  0.2365,  0.5613,  0.7738],
        [ 0.7318, -0.4043, -0.2384,  0.4409],
        [ 0.3368, -0.3591, -0.8219, -0.3415]], grad_fn=<AddBackward0>)
tensor([-0.2991, -0.5083, -0.3425, -0.5579])
tensor([[-0.0882,  0.1290,  0.0674,  0.1725],
        [-0.0785, -0.1051,  0.1533, -0.1014],
        [ 0.0451, -0.0962,  0.1815,  0.0388],
        [ 0.1052, -0.3262,  0.0703, -0.1310]])
tensor([-0.2991, -0.5083, -0.3425, -0.5579])


In [77]:
for l in m_list[:-1] :
    print(l.weight)

for l in m_list2[:-1] :
    print(l.weight)

Parameter containing:
tensor([[-0.6171,  0.7107,  0.9184, -0.5559],
        [ 0.6047, -0.8292, -0.7960, -0.5624],
        [-0.6567,  0.8460,  0.7900, -0.7990],
        [-0.8842,  0.9835,  0.5539,  0.8947]], requires_grad=True)
Parameter containing:
tensor([2.0000, 2.0000, 2.0000, 2.0000], requires_grad=True)
Parameter containing:
tensor([[ 1.3829, -1.2893, -1.0816, -0.5559],
        [ 0.6047,  1.1708, -0.7960,  1.4376],
        [-0.6567,  0.8460, -1.2100, -0.7990],
        [-0.8842,  0.9835, -1.4461,  0.8947]], requires_grad=True)
Parameter containing:
tensor([2.0000, 2.0000, 2.0000, 2.0000], requires_grad=True)


In [78]:
m_list[0].weight

Parameter containing:
tensor([[-0.6171,  0.7107,  0.9184, -0.5559],
        [ 0.6047, -0.8292, -0.7960, -0.5624],
        [-0.6567,  0.8460,  0.7900, -0.7990],
        [-0.8842,  0.9835,  0.5539,  0.8947]], requires_grad=True)

In [79]:
list(s.modules())

[SuperNetwork(
   (lin): Linear(in_features=4, out_features=4, bias=True)
   (bn): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (activation): ReLU()
 ),
 Linear(in_features=4, out_features=4, bias=True),
 BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 ReLU()]